This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_conversion/statsmodels).

## Introduction

Organizations can deploy a Machine Learning (ML) model based on the [statsmodels](https://www.statsmodels.org/stable/index.html) directly into Wallaroo through the following process.  This conversion process transforms the model into an open format that can be run across different frameworks at compiled C-language speeds.

This example provides the following:

* `train-statsmodel.ipynb`: A sample Jupyter Notebook that trains a sample model.  The model predicts how many bikes will be rented on each of the next 7 days, based on the previous 7 days' bike rentals, temperature, and wind speed.  Additional files to support this example are:
  * `day.csv`: Data used to train the sample `statsmodel` example.
  * `infer.py`: The inference script that is part of the `statsmodel`.
* `convert-statsmodel-tutorial.ipynb`: A sample Jupyter Notebook that demonstrates how to upload, convert, and deploy the `statsmodel` example into a Wallaroo instance.    Additional files to support this example are:
  * `bike_day_model.pkl`: A `statsmodel` ML model trained from the `train-statsmodel.ipynb` Notebook.

    **IMPORTANT NOTE:** The `statsmodel` ML model is composed of two parts that are contained in the .pkl file:

    * The pickled Python runtime expects a dictionary with two keys: `model` and `script`:

      * `model`—the pickled model, which will be automatically loaded into the python runtime with the name 'model'
      * `script`—the text of the python script to be run, in a format similar to the existing python script steps (i.e. defining a wallaroo_json method which operates on the data).  In this cae, the file `infer.py` is the script used.

  * `bike_day_eval.json`: Evaluation data used to test the model's performance.

## Steps

The following steps will perform the following:
    
1. Upload the `statsmodel` ML model `bike_day_model.pkl` into a Wallaroo.
2. Deploy the model into a pipeline.
3. Run a test inference.
4. Undeploy the pipeline.

### Import Libraries

The first step is to import the libraries that we will need.

In [35]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [36]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR PREFIX"


wallarooPrefix = "docreleasetest"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [38]:
workspace_name = 'statsmodelworkspace'
pipeline_name = 'statsmodelpipeline'
model_name = 'bikedaymodel'
model_file_name = 'bike_day_model.pkl'

## Set the Workspace and Pipeline

This sample code will create or use the existing workspace `bike-day-workspace` as the current workspace.

In [39]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,statsmodelpipeline
created,2023-05-02 17:52:47.081413+00:00
last_updated,2023-05-02 17:52:47.081413+00:00
deployed,(none)
tags,
versions,5139657b-3ec3-4918-9152-3ee5bf8f1873
steps,


### Upload Pickled Package Statsmodel Model

Upload the statsmodel stored into the pickled package `bike_day_model.pkl`.  See the Notebook `train-statsmodel.ipynb` for more details on creating this package.

Note that this package is being specified as a `python` configuration.

In [40]:
file_name = "bike_day_model.pkl"

bike_day_model = wl.upload_model(model_name, model_file_name).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

In [41]:
pipeline.add_model_step(bike_day_model)

name,statsmodelpipeline
created,2023-05-02 17:52:47.081413+00:00
last_updated,2023-05-02 17:52:47.081413+00:00
deployed,(none)
tags,
versions,5139657b-3ec3-4918-9152-3ee5bf8f1873
steps,


In [42]:
pipeline.deploy()

name,statsmodelpipeline
created,2023-05-02 17:52:47.081413+00:00
last_updated,2023-05-02 17:52:50.438574+00:00
deployed,True
tags,
versions,"6143680b-c2f3-4642-952f-ece29f324f82, 5139657b-3ec3-4918-9152-3ee5bf8f1873"
steps,bikedaymodel


In [43]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.1.54',
   'name': 'engine-66b7f68c8f-4v9lz',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'statsmodelpipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'bikedaymodel',
      'version': 'a654ded7-d3f1-4727-9782-4a47aed54a94',
      'sha': 'cca8a0e8ad6de45a1af42000ea182a2b70bd78390d2418290df1ceea0d43317d',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.4.31',
   'name': 'engine-lb-584f54c899-zbk9l',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

Perform an inference from the evaluation data JSON file `bike_day_eval.json`.

In [44]:
results = pipeline.infer_from_file('bike_day_eval.json', data_format="custom-json")

display(results)

[{'forecast': [1882.3784555157672,
   2130.607915701861,
   2340.84005381799,
   2895.754978552066,
   2163.657515565616,
   1509.1792126509536,
   2431.1838923957016]}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [45]:
pipeline.undeploy()

name,statsmodelpipeline
created,2023-05-02 17:52:47.081413+00:00
last_updated,2023-05-02 17:52:50.438574+00:00
deployed,False
tags,
versions,"6143680b-c2f3-4642-952f-ece29f324f82, 5139657b-3ec3-4918-9152-3ee5bf8f1873"
steps,bikedaymodel
